In [2]:
import numpy as np
import pandas as pd

In [5]:
# Hagan's SABR formula
def SABR(alpha, beta, nu, rho, F, K, T):

    eps_SABR = 0.0001
    
    V = (F * K)**((1.0 - beta) / 2.0)
    logFK = np.log(F / K)
    
    tmp1 = nu * nu * (2.0 - 3.0 * rho * rho) / 24.0
    tmp1 += rho * beta * nu * alpha / (V * 4.0)
    tmp1 += (1.0 - beta) * (1.0 - beta) * alpha * alpha / ((V**2) * 24.0)
    tmp1 = alpha * (1.0 + (tmp1 * T))
    
    tmp2 = V * (1.0 + np.power(logFK * (1.0 - beta), 2.0) / 24.0 + np.power(logFK * (1.0 - beta), 4.0) / 1920.0)
    
    z = nu / alpha  * V * logFK
    
    def chi(z):
        
        eps_SABR = 0.0001
        
        tmp1 = np.sqrt(1.0 - 2.0 * rho * z + z * z)
        np.where(abs(z) < eps_SABR, z, z)
        
        z = np.where((tmp1 + z - rho > 0.0),
                     np.log((tmp1 + z - rho) / (1.0 - rho)), np.log((1.0 + rho) / (tmp1 - (z - rho))))
        
        return z
    
    chi_z = chi(z)
    
    vol = np.where(abs(F - K) > eps_SABR, tmp1 / tmp2 * z / chi_z, tmp1 / np.power(F, 1.0 - beta))
    
    return vol        

In [18]:
nSamples = 1000000
rngSeed = 42
rng = np.random.RandomState(rngSeed)

# Generate
beta = rng.uniform(0.10, 0.80, (nSamples, 1))
nu = rng.uniform(0.20, 0.80, (nSamples, 1))
rho = rng.uniform(-0.40, 0.40, (nSamples, 1))
F = rng.uniform(0.02, 0.04, (nSamples, 1))
alpha = rng.uniform(0.05, 0.25, (nSamples, 1)) / F**(beta - 1.0)
K = rng.uniform(0.01, 0.06, (nSamples, 1))
T = rng.uniform(1.0 / 12.0, 5.0, (nSamples, 1))
IV = SABR(alpha, beta, nu, rho, F, K, T) 

# Save into csv file
data = pd.DataFrame({'alpha': alpha[:,0], 'beta': beta[:,0], 'nu': nu[:,0], 'rho': rho[:,0],
                     'F': F[:,0], 'K': K[:,0], 'T': T[:,0], 'IV': IV[:,0]})

data = data.drop(data[data.IV > 1.5].index) # Remove high vols

data.to_csv("Samples_SABR.csv", sep=',', index = False)